In [ ]:
import json, os
import pandas as pd
import base64
import datetime
import urllib.request

In [ ]:
last_file_name = ""
LAST_FILE_KEY = "last_sent_file_to_ES"
with open("config.json", "r") as f:
    file_content = f.read()

In [ ]:
file_content_json = json.loads(file_content)
file_content_json[LAST_FILE_KEY]
username = file_content_json['user']
password = file_content_json['pass']

In [ ]:
def parse_file_name(file_name):
    if type(file_name) == str:
        file_name_, extension = file_name.split('.')
        if 'csv' == extension:
            date_str, coin_type = file_name_.split('_')
            if len(date_str) == 8:
                year = date_str[:4]
                month = date_str[4:6]
                day = date_str[6:]
                return True, year, month, day, coin_type, extension
            return False, None, None, None, coin_type, extension
        else:
            return False, None, None, None, None, extension
    else:
        return False,None, None, None, None, None

In [ ]:
suc, year, month, day, coin_type, extension = parse_file_name(file_content_json[LAST_FILE_KEY])
last_date = datetime.datetime(year=int(year), month=int(month), day=int(day))
start_date = last_date

In [ ]:
while start_date < datetime.datetime.now():
    ## convert date str to date then add one day
    delta = datetime.timedelta(days=1)
    start_date += delta


    year = start_date.year
    month = "{0:02d}".format(start_date.month)
    day = "{0:02d}".format(start_date.day)
    file_name = f"{year}{month}{day}_{coin_type}.{extension}"
    path = os.path.join('resources', file_name)
    if os.path.exists(path) == False:
        print(f"file not exist. stop the execution. {path}")
        continue
    
    ## read csv as DataFrame
    contents = pd.read_csv(path)
    
    ## Convert DataFrame to desired json format
    scheme_template = {
        "price":None,
        "doc_id":None,
        "provider":None,
        "type":None,
        "order":None,
        "amount":None,
        "scheme":None,
        "@timestamp":None
    }

    last_timestamp = datetime.datetime.now()
    same_time_counter = 0

    for index in range(0, len(contents)):
        order_log = scheme_template.copy()
        order_log["provider"]= 'gmo'
        order_log["type"] = 'btc_jpy'#buy_sell
        order_log["scheme"] = 'order'#board or order

        content = contents.iloc[index]
        doctime = datetime.datetime.strptime(content.timestamp, "%Y-%m-%d %H:%M:%S.%f")
        if last_timestamp == doctime:
            same_time_counter += 1
        else:
            last_timestamp = doctime
            same_time_counter = 0
        doc_id = str(doctime.timestamp()).replace('.', '') + "{0:03d}".format(same_time_counter)
        order_log["price"] = content.price
        order_log["amount"] = str(content["size"])
        order_log["doc_id"] = doc_id
        order_log["order"] = content.side
        order_log["@timestamp"] = content.timestamp.replace(' ', 'T') + "+09:00"

        ## Send json to ES with creds -- TODO: read ES creds from env file.
        body = json.dumps(order_log)
        url = "http://192.168.100.144:5400"

        req = urllib.request.Request(url)
        req.add_header('Content-Type', 'application/json; charset=utf-8')
        jsondataasbytes = body.encode('utf-8')   # needs to be bytes
        req.add_header('Content-Length', len(jsondataasbytes))
        creds = base64.b64encode(('f{username}:{password}').encode('utf-8'))
        req.add_header('Authorization', f'Basic {creds}')
        ## Add error handling and retry logic 
        response = urllib.request.urlopen(req, jsondataasbytes)

In [ ]:
# When timeout is happend, try again
start_date = start_date - delta

In [ ]:
contents
#maybe timestamp is JST?